In [ ]:
import numpy as np
import sys
np.set_printoptions(linewidth =np.inf,threshold=np.inf)
from queue import Queue
import threading
import copy
import pandas as pd
 
    
class Robot:
    def __init__(self, sequence):
        self.sequence = sequence
        
        self.direction_int_mapping = {"up": 0, "right": 1, "down": 2, "left": 3}
        self.direction_symbole_mapping = {"up": "^", "right": ">", "down": "v", "left": "<"}
        self.int_direction_mapping = {val: key for key, val in self.direction_int_mapping.items()}
        self.direction_correspondance = {"up": (-1, 0), "right": (0, 1), "down": (1, 0), "left": (0, -1)}
        
        self.reset()
    
    def reset(self):
        self.position = (0, 0)
        self.canevas = {self.position: 0}
        self.direction = "up"
        self.input_val = Queue(1)
        self.output_val = Queue(2)
    
    def part_1(self, print_map=False):
        self.reset()
        self._start_thread(print_map)
        
        return len(self.canevas)
    
    def part_2(self, print_map=True):
        self.reset()
        self.canevas[self.position] = 1
        self._start_thread(print_map)
        
        print(self._map())
    
    def _map(self):
        copy_canevas = copy.copy(self.canevas)
        copy_canevas.update({self.position: 0})
        
        min_x = min(copy_canevas, key=lambda k: k[0])[0]
        max_x = max(copy_canevas, key=lambda k: k[0])[0]
        min_y = min(copy_canevas, key=lambda k: k[1])[1]
        max_y = max(copy_canevas, key=lambda k: k[1])[1]

        offset = (abs(min_x), abs(min_y))
        canevas = np.full((max_x - min_x + 1, max_y - min_y + 1), ".", dtype=str)
        for index, value in copy_canevas.items():
            if value == 1:
                canevas[tuple([sum(e) for e in zip(offset, index)])] = "#"
        
        canevas[tuple([sum(e) for e in zip(offset, self.position)])] = self.direction_symbole_mapping[self.direction]
        return canevas
    
    def _process(self, print_map=False):
        process = True
        while(process):
            self.input_val.put(self.canevas.get(self.position, 0))
            
            color = self.output_val.get()
            
            if color == -1:
                process = False
                break
            else:
                self.canevas.update({self.position: color})

            move = self.output_val.get()

            if move == 0:
                self._rotate_left()
            elif move == 1:
                self._rotate_right()
            else:
                process = False
                break
            
            self._move()
            
            if print_map:
                print(self._map())
                print()
            
    
    def _move(self):
        self.position = tuple([sum(i) for i in zip(self.position, self.direction_correspondance[self.direction])])
    
    def _rotate_left(self):
        self.direction = self.int_direction_mapping[(self.direction_int_mapping[self.direction] - 1) % len(self.direction_int_mapping)]
        
    def _rotate_right(self):
        self.direction = self.int_direction_mapping[(self.direction_int_mapping[self.direction] + 1) % len(self.direction_int_mapping)]
    
    def _start_thread(self, print_map=False):
        threads = []
        # Create the threads
        thread_process = threading.Thread(target=self._process, args=(print_map,))
        thread_seq = threading.Thread(target=self._seq_reader)
        
        # start the threads
        thread_process.start()
        thread_seq.start()
        
        # wait for the threads to end
        thread_seq.join()
        print("thread_seq fini!")
        self.output_val.put(-1)  # Condition to end the process()
        thread_process.join()
        print("thread_process fini!")
        # retrive the value
        

    def _seq_reader(self):
        def read_mod(offset):
            return mods[offset] if 0 <= offset < len(mods) else 0
    
        def read_val(index, offset):
            val = seq.get(index + offset, 0)

            mod = read_mod(offset)
            if mod == 0:
                return seq.get(val, 0)
            elif mod == 1:
                return val
            elif mod == 2:
                return seq.get(relative_base + val, 0)

        def write_val(index, new_value, offset):
            val = seq.get(index + offset, 0)

            mod = read_mod(offset)
            if mod == 0:
                seq.update({val: new_value})
            elif mod == 2:
                seq.update({relative_base + val: new_value})

        relative_base = 0
        seq = {index:val for index, val in enumerate(self.sequence)}
        go = True
        index = 0
        
        while(go):
            opcode = seq[index]
            val = opcode % 100
            mods = list(map(int, str(opcode)))[:-2]
            mods.reverse()
            index += 1

            if val in [1, 2, 7, 8]:
                op1 = read_val(index, 0)
                op2 = read_val(index, 1)
                if val == 1:
                    res = op1 + op2
                elif val == 2:
                    res = op1 * op2
                elif val == 7:
                    res = 1 if op1 < op2 else 0
                elif val == 8:
                    res = 1 if op1 == op2 else 0

                write_val(index, res, 2)
                index += 3

            elif val == 3:
                input_val = self.input_val.get()
                #input_val = int(input("input your value: "))
                write_val(index, input_val, 0)

                index += 1

            elif val in [4, 9]:
                op1 = read_val(index, 0)
                if val == 4:
                    #print("output value: {}".format(op1))
                    self.output_val.put(op1)
                elif val == 9:
                    relative_base += op1

                index += 1

            elif val in [5, 6, 9]:
                op1 = read_val(index, 0)

                if val == 5 and op1 or val == 6 and not op1:
                    index = read_val(index, 1)
                else:
                    index += 2

            elif val == 99:
                print("seq finit!")
                go = False
                continue

            else:
                raise ValueError("RIP")

In [ ]:
seq = [3,8,1005,8,309,1106,0,11,0,0,0,104,1,104,0,3,8,102,-1,8,10,101,1,10,10,4,10,1008,8,1,10,4,10,1001,8,0,29,3,8,102,-1,8,10,101,1,10,10,4,10,1008,8,0,10,4,10,102,1,8,51,3,8,102,-1,8,10,1001,10,1,10,4,10,108,0,8,10,4,10,1002,8,1,72,1,1104,8,10,2,1105,15,10,2,1106,0,10,3,8,1002,8,-1,10,1001,10,1,10,4,10,1008,8,1,10,4,10,101,0,8,107,3,8,102,-1,8,10,1001,10,1,10,4,10,108,1,8,10,4,10,101,0,8,128,2,6,8,10,3,8,102,-1,8,10,101,1,10,10,4,10,1008,8,0,10,4,10,102,1,8,155,1006,0,96,2,108,10,10,1,101,4,10,3,8,1002,8,-1,10,101,1,10,10,4,10,1008,8,0,10,4,10,1002,8,1,188,2,1,5,10,3,8,102,-1,8,10,101,1,10,10,4,10,1008,8,0,10,4,10,102,1,8,214,2,6,18,10,1006,0,78,1,105,1,10,3,8,1002,8,-1,10,1001,10,1,10,4,10,1008,8,1,10,4,10,102,1,8,247,2,103,8,10,2,1002,10,10,2,106,17,10,1,1006,15,10,3,8,102,-1,8,10,101,1,10,10,4,10,1008,8,1,10,4,10,101,0,8,285,1,1101,18,10,101,1,9,9,1007,9,992,10,1005,10,15,99,109,631,104,0,104,1,21102,387507921664,1,1,21102,1,326,0,1106,0,430,21102,932826591260,1,1,21102,337,1,0,1106,0,430,3,10,104,0,104,1,3,10,104,0,104,0,3,10,104,0,104,1,3,10,104,0,104,1,3,10,104,0,104,0,3,10,104,0,104,1,21101,206400850983,0,1,21101,0,384,0,1105,1,430,21102,3224464603,1,1,21102,395,1,0,1106,0,430,3,10,104,0,104,0,3,10,104,0,104,0,21102,838433657700,1,1,21102,418,1,0,1106,0,430,21101,825012007272,0,1,21101,429,0,0,1106,0,430,99,109,2,21202,-1,1,1,21101,40,0,2,21101,461,0,3,21102,1,451,0,1105,1,494,109,-2,2105,1,0,0,1,0,0,1,109,2,3,10,204,-1,1001,456,457,472,4,0,1001,456,1,456,108,4,456,10,1006,10,488,1102,1,0,456,109,-2,2106,0,0,0,109,4,1202,-1,1,493,1207,-3,0,10,1006,10,511,21101,0,0,-3,21202,-3,1,1,21201,-2,0,2,21102,1,1,3,21102,1,530,0,1106,0,535,109,-4,2106,0,0,109,5,1207,-3,1,10,1006,10,558,2207,-4,-2,10,1006,10,558,22101,0,-4,-4,1106,0,626,22102,1,-4,1,21201,-3,-1,2,21202,-2,2,3,21101,0,577,0,1106,0,535,22102,1,1,-4,21101,1,0,-1,2207,-4,-2,10,1006,10,596,21102,0,1,-1,22202,-2,-1,-2,2107,0,-3,10,1006,10,618,21201,-1,0,1,21102,618,1,0,105,1,493,21202,-2,-1,-2,22201,-4,-2,-4,109,-5,2105,1,0]

r = Robot(seq)


In [ ]:
r.part_1()

In [ ]:
array = r.part_2()

print(array)


In [ ]:
one = np.ones((2, 2))

In [ ]:
empty = np.empty((2, 2), dtype=str)

In [ ]:
three = np.where(one==1, 3, one)

In [ ]:
one

In [ ]:
three

In [ ]:
abs((1, -1))